In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install finance-datareader
!pip install neuralprophet[live]

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 155 kB 9.1 MB/s 
  Attempting uninstall: holidays
    Found existing installation: holidays 0.10.5.2
    Uninstalling holidays-0.10.5.2:
      Successfully uninstalled holidays-0.10.5.2


In [3]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from fbprophet import Prophet

from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed 

import warnings
import datetime
import random
import tensorflow as tf
warnings.filterwarnings("ignore")

# PUBLIC

In [4]:
path = '/content/drive/MyDrive/stock_2/'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [5]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/train')

In [6]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [7]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sample_submission = sample_submission[['Day']]
sample_submission['Day'] =pd.to_datetime(sample_submission['Day'])

In [8]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/train/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [03:02<00:00,  2.03it/s]


In [9]:
# model = NeuralProphet(
#     n_forecasts=5,
#         n_lags=370,
#         yearly_seasonality=False,
#         weekly_seasonality=False,
#         daily_seasonality=False,
#         batch_size=64,
#         epochs=100,
#         learning_rate=1.0,
# )

# metric = model.fit(df_000060, freq='D', valid_p=0.1, epochs=100)

# future = model.make_future_dataframe(df_000060, periods=7)
# forecast = model.predict(future)

In [10]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=60,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

INFO - (NP.forecaster._handle_missing_data) - 95 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 95 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:02<00:00, 44.62it/s, L1Loss=0.0392, MAE=636, MSE=8e+5, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 32 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 32 NaN values in column y were auto-imputed.
INFO - (NP.forecaster._handle_missing_data) - 95 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 95 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override 

In [14]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][92])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][93])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][94])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][95])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][96])

In [15]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sub_public = sample_submission[0:5]
sub_private = sample_submission[5:10]

In [16]:
for i in sorted(companyList):
    sub_public[i] = locals()['list_{}'.format(i)]

In [18]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/test')

In [19]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [20]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/test/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [04:02<00:00,  1.53it/s]


In [21]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=60,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

Epoch[100/100]: 100%|██████████| 100/100 [00:03<00:00, 26.77it/s, L1Loss=0.0602, MAE=127, MSE=2.69e+4, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 29 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 29 NaN values in column y were auto-imputed.
INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:03<00:00, 27.08it/s, L1Loss=0.0776, MAE=2.39e+3, MSE=1.05e+7, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 29 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 29 NaN values in column y were auto-imputed.
INFO - (N

In [22]:
forecast_000060

,ds,y,yhat1,residual1,yhat2,residual2,yhat3,residual3,yhat4,residual4,yhat5,residual5,ar1,ar2,ar3,ar4,ar5,trend,season_weekly
0,2021-08-30,24600.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2021-08-31,26550.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2021-09-01,29200.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2021-09-02,29700.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2021-09-03,29200.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2021-11-27,NaN,29574,NaN,30230.8,NaN,28803.3,NaN,27904.2,NaN,28889.2,NaN,17046.8,17703.6,16276.1,15377,16362,12604.6,-77.3461
90,2021-11-28,NaN,None,NaN,30654.9,NaN,29629.5,NaN,28664.9,NaN,29090.9,NaN,None,18198.9,17173.5,16209,16634.9,12577.7,-121.714
91,2021-11-29,NaN,None,NaN,None,NaN,30526.9,NaN,29291.9,NaN,29165.9,NaN,None,None,18134.3,16899.3,16773.3,12550.8,-158.163
92,2021-11-30,NaN,None,NaN,None,NaN,None,NaN,30716.7,NaN,29343.3,NaN,None,None,None,18133.6,16760.1,12523.9,59.2843


In [23]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][89])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][90])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][91])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][92])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][93])

In [24]:
for i in sorted(companyList):
    sub_private[i] = locals()['list_{}'.format(i)]

In [25]:
sub_total = pd.concat([sub_public, sub_private])

In [27]:
sub_total.to_csv(path + 'neural_prophet_lag_60.csv', index=False)